In [1]:
from hartufo import HutubsPlane, ItaPlane
import hartufo
from pathlib import Path
import matplotlib.pyplot as plt
import scipy.signal
import soundfile as sf
import numpy as np
import librosa
import random
import torchaudio

In [2]:
base_dir = Path('/nas/home/jalbarracin/datasets/HUTUBS')

In [5]:
plane = 'horizontal'
domain = 'time'
side = 'both'

ds = HutubsPlane(base_dir / 'HUTUBS', plane, domain, side, download=False)
samplerate=44100

Preprocessing hrir: 100%|██████████| 1/1 [00:00<00:00, 337.35it/s]


In [4]:
len(ds)

86

In [5]:
for j in range(len(ds)):
    p = ds[j]
    for x in range(p['features'].shape[0]):
        left_data = np.asarray(p['features'][x,:])
        sf.write(f'/nas/home/jalbarracin/datasets/hrir_l/pp{ds.available_subject_ids[j]}_HRIRs_measured_{x}_l.wav', left_data, samplerate)

In [3]:
import os
import pandas as pd
import re

directory = '/nas/home/jalbarracin/datasets/hrir_st'

# Initialize lists to store extracted information
file_names = []
subject_ids = []
azimuths = []

# Define a function to extract subject ID and azimuth from file name
def extract_info(file_name):
    match = re.match(r"pp(\d+)_HRIRs_measured_(\d+).wav", file_name)
    if match:
        subject_id = int(match.group(1))
        azimuth = int(match.group(2))
        return subject_id, azimuth
    else:
        return None

# Loop through files in the directory
for file_name in os.listdir(directory):
    file_path = os.path.join(directory, file_name)
    
    # Check if the file is a WAV file
    if file_name.endswith(".wav"):
        file_names.append(file_name)
        
        # Extract subject ID and azimuth
        info = extract_info(file_name)
        if info:
            subject_id, azimuth = info
            subject_ids.append(subject_id)
            azimuths.append(azimuth)

# Create a DataFrame with the extracted information
data = {'file_name': file_names, 'subject_id': subject_ids, 'azimuth': azimuths}
df = pd.DataFrame(data)

# Optional: Create a label column as discussed in previous responses
df['label'] = df.apply(lambda row: f"Subject{row['subject_id']}_{row['azimuth']}", axis=1)
df.to_csv('labeled_data.csv', index=False)